# AQC per il problema dello zaino

In [1]:
import math
import pandas as pd
import dimod
import sympy as sp
from sympy.abc import m, M, c, lamda
import dwave.system as dw

/Users/giulia/tesinaOC/ocean/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


### Caricamento dati
Lettura di un dataset dalla cartella *data*. Per motivi di tempi di esecuzioni e capacità delle macchine i test sono stati fatti sui dataset più piccoli, come *small* e *very_small*.
Si stampa il dataset, composto dalla colonna dei **profitti** e la colonna dei **pesi**.

In [2]:
data = pd.read_csv('data/very_small.csv', names=['p', 'w'])
data.index = range(1, len(data) + 1)

data

,p,w
1,3,1
2,4,3
3,2,2


Si setta la capacità dello zaino manualmente per motivi di praticità: era difficile inserirla nel dataset in formato csv

In [3]:
num_items = len(data)
capacity = int(0.8 * sum(data['w'])) # Fissa capacità dello zaino all'80% della somma dei pesi degli elementi

print(f'c = {capacity}')

c = 4


### Prima formulazione
L'implementazione segue quanto descritto nella tesina.
Si introducono delle variabili di slack $y_i$ per rappresentare tutti i possibili valori di riempimento dello zaino.
Si è usata la libreria **sympy** per scrivere le formule ed effettuare in modo meccanico i calcoli, sotto seguono dei procedimenti per definire variabili e indici per la scrittura della formula.

In [4]:
j = sp.Idx('j')         # Indici per le sommatorie
n = sp.Idx('n')
X = sp.IndexedBase('x') # Vettori variabili oggetti
P = sp.IndexedBase('p') # Vettori variabili profitto
W = sp.IndexedBase('w') # Vettori variabili pesi
Y = sp.IndexedBase('y') # Vettori variabili di slack

syms = [X[k] for k in range(1, num_items + 1)] + [Y[k] for k in range(1, capacity + 1)]

In [5]:
first_f = -sp.Sum(P[j] * X[j], (j, 1, m)) + \
          lamda * ((1 - sp.Sum(Y[n], (n, 1, c)))**2 +
                   (sp.Sum(n * Y[n], (n, 1, c)) - sp.Sum(W[j] * X[j], (j, 1, m)))**2)
first_f

lamda*((1 - Sum(y[n], (n, 1, c)))**2 + (Sum(n*y[n], (n, 1, c)) - Sum(w[j]*x[j], (j, 1, m)))**2) - Sum(p[j]*x[j], (j, 1, m))

Una volta definita la formula vengono fatte le sostituzioni con i valori effettivi. Il $.doit()$ serve per svolgere alcune sostituzioni e passaggi prima di procedere con il resto, serve per motivi di ottimizzazione delle prestazioni: far fare tutte le sostituzioni insieme può portare a problemi.

In [6]:
first_f = first_f.subs([(m, num_items), (c, capacity)]).doit() \
                 .subs([(P[i], data['p'][i]) for i in range(1, num_items + 1)]) \
                 .subs([(W[i], data['w'][i]) for i in range(1, num_items + 1)]) \
                 .subs(lamda, 1 + data['p'].max())
first_f

5*(-y[1] - y[2] - y[3] - y[4] + 1)**2 + 5*(-x[1] - 3*x[2] - 2*x[3] + y[1] + 2*y[2] + 3*y[3] + 4*y[4])**2 - 3*x[1] - 4*x[2] - 2*x[3]

Si espande la funzione. Dal momento che siamo in un dominio binario si possono sostituire tutte le $x^n$ con la loro base $x$

In [7]:
first_f = sp.expand(first_f).replace(lambda x: x.is_Pow, lambda x: x.base); first_f

30*x[1]*x[2] + 20*x[1]*x[3] - 10*x[1]*y[1] - 20*x[1]*y[2] - 30*x[1]*y[3] - 40*x[1]*y[4] + 2*x[1] + 60*x[2]*x[3] - 30*x[2]*y[1] - 60*x[2]*y[2] - 90*x[2]*y[3] - 120*x[2]*y[4] + 41*x[2] - 20*x[3]*y[1] - 40*x[3]*y[2] - 60*x[3]*y[3] - 80*x[3]*y[4] + 18*x[3] + 30*y[1]*y[2] + 40*y[1]*y[3] + 50*y[1]*y[4] + 70*y[2]*y[3] + 90*y[2]*y[4] + 15*y[2] + 130*y[3]*y[4] + 40*y[3] + 75*y[4] + 5

In [8]:
first_Q = pd.DataFrame(0, index=syms, columns=syms)

first_p = sp.Poly(first_f)  # Viene interpretato come un polinomio

for sym1 in syms:
    for sym2 in syms[syms.index(sym1):]:
        first_Q.loc[sym1, sym2] = int(first_p.coeff_monomial(sym1 if sym1 == sym2 else sym1*sym2))

first_Q

,x[1],x[2],x[3],y[1],y[2],y[3],y[4]
x[1],2,30,20,-10,-20,-30,-40
x[2],0,41,60,-30,-60,-90,-120
x[3],0,0,18,-20,-40,-60,-80
y[1],0,0,0,0,30,40,50
y[2],0,0,0,0,15,70,90
y[3],0,0,0,0,0,40,130
y[4],0,0,0,0,0,0,75


### Approccio brute force
Viene eseguito un metodo di **brute force** nel quale vengono calcolate tutte le soluzioni possibili, che sono poi ordinate in senso non decrescente di valore energetico.
Per dataset grossi questo approccio non funziona, in quanto eccede i limiti di memoria utilizzabili.

In [9]:
dimod.ExactSolver().sample_qubo(first_Q).to_pandas_dataframe().sort_values(by='energy')

,0,1,2,3,4,5,6,energy,num_occurrences
125,1,1,0,0,0,0,1,-12.0,1
57,1,0,1,0,0,1,0,-10.0,1
101,1,1,1,0,1,0,1,-9.0,1
60,0,1,0,0,0,1,0,-9.0,1
14,1,0,0,1,0,0,0,-8.0,1
...,...,...,...,...,...,...,...,...,...
79,0,0,0,1,0,1,1,335.0,1
87,0,0,1,1,1,1,1,358.0,1
95,0,0,0,0,1,1,1,420.0,1
81,1,0,0,1,1,1,1,442.0,1


### Approccio Simulated Annealing
Viene eseguito un metodo di **simulated annealing** nel quale si approssima il valore di minimo globale della funzione. Al pari di altre euristiche, è un valido metodo per ottenere buone soluzioni in una quantità ragionevole di tempo.

In [10]:
dimod.SimulatedAnnealingSampler().sample_qubo(first_Q).to_pandas_dataframe().sort_values(by='energy')

,0,1,2,3,4,5,6,energy,num_occurrences
5,0,1,0,0,0,1,0,-9.0,1
8,0,1,0,0,0,1,0,-9.0,1
9,0,1,0,0,0,1,0,-9.0,1
2,1,0,0,1,0,0,0,-8.0,1
3,1,0,0,1,0,0,0,-8.0,1
1,0,1,1,0,1,1,0,-6.0,1
6,0,1,1,1,0,0,1,-6.0,1
7,0,1,1,1,0,0,1,-6.0,1
0,1,1,1,0,0,1,1,-4.0,1
4,1,1,1,0,0,1,1,-4.0,1


### Seconda formulazione
L'implementazione segue quanto descritto nella tesina.
Il numero di variabili è notevolmente ridotto in questo approccio, rendendolo più semplice ed efficiente.

In [11]:
i = sp.Idx('i')
j = sp.Idx('j')
n = sp.Idx('n')
X = sp.IndexedBase('x')
P = sp.IndexedBase('p')
W = sp.IndexedBase('w')
Y = sp.IndexedBase('y')

syms = [X[k] for k in range(1, num_items + 1)] + [Y[k] for k in range(0, math.floor(math.log(capacity, 2)) + 1)]

In [12]:
second_f = -sp.Sum(P[j] * X[j], (j, 1, m)) + \
           lamda * (((c - (2**M - 1)) * Y[M] + sp.Sum(2**i * Y[i], (i, 0, M - 1)) -
                     sp.Sum(W[j] * X[j], (j, 1, m)))**2)
second_f

lamda*((-2**M + c + 1)*y[M] + Sum(2**i*y[i], (i, 0, M - 1)) - Sum(w[j]*x[j], (j, 1, m)))**2 - Sum(p[j]*x[j], (j, 1, m))

In [13]:
second_f = second_f.subs([(m, num_items), (c, capacity), (M, math.floor(math.log(capacity, 2)))]).doit() \
                   .subs([(P[i], data['p'][i]) for i in range(1, num_items + 1)]) \
                   .subs([(W[i], data['w'][i]) for i in range(1, num_items + 1)]) \
                   .subs(lamda, 1 + data['p'].max())

second_f

5*(-x[1] - 3*x[2] - 2*x[3] + y[0] + 2*y[1] + y[2])**2 - 3*x[1] - 4*x[2] - 2*x[3]

In [14]:
second_f = second_f.expand().replace(lambda x: x.is_Pow, lambda x: x.base); second_f

30*x[1]*x[2] + 20*x[1]*x[3] - 10*x[1]*y[0] - 20*x[1]*y[1] - 10*x[1]*y[2] + 2*x[1] + 60*x[2]*x[3] - 30*x[2]*y[0] - 60*x[2]*y[1] - 30*x[2]*y[2] + 41*x[2] - 20*x[3]*y[0] - 40*x[3]*y[1] - 20*x[3]*y[2] + 18*x[3] + 20*y[0]*y[1] + 10*y[0]*y[2] + 5*y[0] + 20*y[1]*y[2] + 20*y[1] + 5*y[2]

In [15]:
second_Q = pd.DataFrame(0, index=syms, columns=syms)

second_p = sp.Poly(second_f)

for sym1 in syms:
    for sym2 in syms[syms.index(sym1):]:
            second_Q.loc[sym1, sym2] = int(second_p.coeff_monomial(sym1 if sym1 == sym2 else sym1*sym2))

second_Q

,x[1],x[2],x[3],y[0],y[1],y[2]
x[1],2,30,20,-10,-20,-10
x[2],0,41,60,-30,-60,-30
x[3],0,0,18,-20,-40,-20
y[0],0,0,0,5,20,10
y[1],0,0,0,0,20,20
y[2],0,0,0,0,0,5


### Approccio brute force
Viene eseguito un metodo di **brute force** nel quale vengono calcolate tutte le soluzioni possibili, che sono poi ordinate in senso non decrescente di valore energetico.
Per dataset grossi questo approccio non funziona, in quanto eccede i limiti di memoria utilizzabili.

In [16]:
dimod.ExactSolver().sample_qubo(second_Q).to_pandas_dataframe().sort_values(by='energy')

,0,1,2,3,4,5,energy,num_occurrences
45,1,1,0,1,1,1,-7.0,1
22,1,0,1,1,1,0,-5.0,1
38,1,0,1,0,1,1,-5.0,1
19,0,1,0,1,1,0,-4.0,1
35,0,1,0,0,1,1,-4.0,1
...,...,...,...,...,...,...,...,...
47,0,0,0,1,1,1,80.0,1
58,1,1,1,0,0,1,116.0,1
10,1,1,1,1,0,0,116.0,1
4,0,1,1,0,0,0,119.0,1


### Approccio Simulated Annealing
Viene eseguito un metodo di **simulated annealing** nel quale si approssima il valore di minimo globale della funzione. Al pari di altre euristiche, è un valido metodo per ottenere buone soluzioni in una quantità ragionevole di tempo.

In [17]:
dimod.SimulatedAnnealingSampler().sample_qubo(second_Q).to_pandas_dataframe().sort_values(by='energy')

,0,1,2,3,4,5,energy,num_occurrences
7,1,1,0,1,1,1,-7.0,1
2,1,0,1,0,1,1,-5.0,1
4,1,0,1,1,1,0,-5.0,1
5,1,0,1,0,1,1,-5.0,1
6,1,0,1,1,1,0,-5.0,1
0,1,0,0,0,0,1,-3.0,1
1,1,0,0,0,0,1,-3.0,1
3,1,0,0,1,0,0,-3.0,1
9,1,0,0,0,0,1,-3.0,1
8,0,0,1,0,1,0,-2.0,1


### Approccio quantistico
Viene eseguito un **approccio quantistico** in cloud con un'architettura fornita gratuitamente da DWave, per questo si ha una capacità di calcolo limitata.
Si può vedere 

In [18]:
#bqm = dimod.BinaryQuadraticModel.from_qubo(second_Q) 
#qsampler = dw.EmbeddingComposite(dw.DWaveSampler()) 
#qsampler.sample(bqm, num_reads=1000).to_pandas_dataframe().sort_values(by='energy')